In [2]:
#Test file load_data

#from src.load_data import load_info #   Import function from file
from src.load_data import multiple_files #Import funtion from file

#Test 1

#df = load_info("data/raw/Musical_Instruments.json.gz")

#df.head()

#Test 2

files = multiple_files()

print(files)

ModuleNotFoundError: No module named 'src'